In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv("cleaned_data.csv",index_col=False)

In [5]:
df = df.drop(columns=['Unnamed: 0'])

In [6]:
df.head()

,name,description,cuisine,course,diet,prep_time,instructions,file_name,ingredients_clean
0,Doddapatre Tambuli Recipe (Karpooravalli Thayi...,Doddapatre Tambuli Recipe is a popular healing...,Indian,Side Dish,Vegetarian,Total in 30 M,"To begin making the Doddapatre Tambuli, first ...",Doddapatre_Tambuli_Recipe_Karuveppilai_Thayir_...,"4 cups Indian borage (Doddapatre) , cleaned an..."
1,Fish Tandoori Recipe,Have you ever tried out fish tandoori? Trust m...,North Indian Recipes,Appetizer,High Protein Non Vegetarian,Total in 80 M,"To begin making the Fish Tandoori recipe, keep...",Fish_Tandoori.jpg,"1 Fish , nicely washed with bones on (I used P..."
2,Arbi Shimla Mirch Sabzi Recipe – Colocasia Cap...,Arbi Shimla Mirch Sabzi Recipe is a variation ...,North Indian Recipes,Main Course,No Onion No Garlic (Sattvic),Total in 55 M,To begin making the Arbi Shimla Mirch Sabzi Re...,Simla_Mirchi_Arbi_Sabzi_Recipe__Capsicum__Colo...,300 grams Colocasia root (Arbi) 1 Green Bell P...
3,Ambur Style Brinjal Curry Recipe,Brinjal Curry is prepared in different ways in...,Indian,Lunch,Vegetarian,Total in 20 M,To begin making the Ambur Style Brinjal Curry ...,Brinjal_curry_for_Biryani.jpg,5 Brinjal (Baingan / Eggplant) 1 Onion 2 Tomat...
4,Mavinakayi Menasinakai Curry Recipe - Raw Man...,Mangalorean Mavinakayi Menasinakai Curry Recip...,Mangalorean,Side Dish,Vegetarian,Total in 35 M,To begin making Mangalorean Mavinakayi Menasin...,Raw_Mango_Coconut_Curry__Mavinakayi_Menasinaka...,"1 cup Mango (Raw) , diced 2 tablespoons Jagger..."


# ml model

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add, RepeatVector


In [2]:
# Constants
DATA_PATH = 'data/'
IMAGE_DIR = DATA_PATH 
CSV_FILE = 'cleaned_data.csv'
IMG_SIZE = 299  # InceptionV3 input size
MAX_CAPTION_LENGTH = 50
VOCAB_SIZE = 10000

# Load CSV
df = pd.read_csv(CSV_FILE)
df['image_path'] = IMAGE_DIR + df['file_name'][1200]

# Load images
def load_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img

# Preprocess captions
df['name'] = df['name'].apply(lambda x: '<start> ' + x + ' <end>')
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<unk>")
tokenizer.fit_on_texts(df['name'])
df['caption_seq'] = tokenizer.texts_to_sequences(df['name'])

# Split dataset
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Load and preprocess images
def load_images(data):
    images = []
    for img_path in data['image_path']:
        images.append(load_image(img_path))
    return np.array(images)

train_images = load_images(train_data)
test_images = load_images(test_data)


# CNN and RNN

In [3]:
# CNN for feature extraction
def create_cnn_model():
    base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
    base_model.trainable = False
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    return Model(inputs, x)

# RNN for captioning
def create_rnn_model():
    inputs = Input(shape=(MAX_CAPTION_LENGTH,))
    x = Embedding(VOCAB_SIZE, 256, mask_zero=True)(inputs)
    x = LSTM(256, return_sequences=True)(x)  # LSTM outputs a sequence
    x = Dropout(0.5)(x)
    x = LSTM(256, return_sequences=True)(x)  # Second LSTM layer also outputs a sequence
    outputs = Dense(VOCAB_SIZE, activation='softmax')(x)  # Predict a word at each step
    return Model(inputs, outputs)

def create_full_model(cnn_model, rnn_model):
    cnn_inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    caption_inputs = Input(shape=(MAX_CAPTION_LENGTH,))
    
    # Extract features from the CNN model
    img_features = cnn_model(cnn_inputs)
    img_features = Dense(256, activation='relu')(img_features)
    
    # Expand the dimensions of img_features to match the RNN output
    img_features = RepeatVector(MAX_CAPTION_LENGTH)(img_features)  # Now shape is (50, 256)

    # Generate caption using the RNN model
    rnn_outputs = rnn_model(caption_inputs)  # Shape is (50, 10000)

    # Ensure the dimensions are compatible
    img_features = Dense(VOCAB_SIZE, activation='relu')(img_features)  # Now shape is (50, 10000)

    # Combine the outputs from CNN and RNN
    combined_outputs = add([img_features, rnn_outputs])

    final_outputs = Dense(VOCAB_SIZE, activation='softmax')(combined_outputs)
    
    return Model(inputs=[cnn_inputs, caption_inputs], outputs=final_outputs)

# Create the models
cnn_model = create_cnn_model()
rnn_model = create_rnn_model()
model = create_full_model(cnn_model, rnn_model)


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

def prepare_training_data(images, data):
    X_images, X_captions, y = [], [], []
    for i, seq in enumerate(data['caption_seq']):
        for j in range(1, len(seq)):
            in_seq, out_word_index = seq[:j], seq[j]  # Use the word index directly
            in_seq = pad_sequences([in_seq], maxlen=MAX_CAPTION_LENGTH)[0]

            # Initialize a zero matrix for the output sequence
            y_seq = np.zeros((MAX_CAPTION_LENGTH, VOCAB_SIZE))

            # Only set the word at the j-th position to 1
            if j < MAX_CAPTION_LENGTH:
                y_seq[j, out_word_index] = 1

            X_images.append(images[i])
            X_captions.append(in_seq)
            y.append(y_seq)
    return np.array(X_images), np.array(X_captions), np.array(y)

X_train_images, X_train_captions, y_train = prepare_training_data(train_images, train_data)

# Train the model
model.fit([X_train_images, X_train_captions], y_train, epochs=2, batch_size=64)
